In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shriman11212234324","key":"57db0c6df36cc67bf1fdcb994f9362e8"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shayalvaghasiya/ntua-prakinson

Dataset URL: https://www.kaggle.com/datasets/shayalvaghasiya/ntua-prakinson
License(s): MIT
100% 2.28G/2.28G [01:00<00:00, 44.8MB/s]
100% 2.28G/2.28G [01:00<00:00, 40.2MB/s]


In [ ]:
!unzip ntua-prakinson.zip

Streaming output truncated to the last 5000 lines.
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range2__022.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range2__023.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range2__024.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range2__025.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range__001.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range__002.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range__003.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range__004.png  
  inflating: ntua-parkinson-dataset-master/PD Patients/Subject68/1.MRI/_MPR_Thick_Range__005.png  
  inflating: ntua-parkinson-dataset-master/PD Patients

In [ ]:
import os
# List the files to check the dataset
os.listdir('ntua-parkinson-dataset-master')


['README.md', 'Non PD Patients', 'PD Patients']

In [ ]:
!pip install qiskit==0.45
!pip install numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 11.2 MB/s eta 0:00:00


In [ ]:
!pip install pennylane torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Define a custom dataset class for MRI images
class ParkinsonsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._load_dataset()

    def _load_dataset(self):
        # Load the dataset by scanning the root directory
        for label, category in enumerate(['Non PD Patients', 'PD Patients']):
            category_path = os.path.join(self.root_dir, category)
            for subject in os.listdir(category_path):
                subject_path = os.path.join(category_path, subject, '1.MRI')
                if os.path.exists(subject_path):
                    for img_file in os.listdir(subject_path):
                        if img_file.endswith('.png'):
                            self.image_paths.append(os.path.join(subject_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Initialize the dataset
dataset = ParkinsonsDataset(root_dir='ntua-parkinson-dataset-master', transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ClassicalCNN(nn.Module):
    def __init__(self):
        super(ClassicalCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Fully connected layer
        self.fc2 = nn.Linear(256, 32)  # Reduced feature vector

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 128 * 16 * 16)  # Flatten the feature map
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # Return the final 32-feature vector


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define a PennyLane quantum device with 1 qubit
dev = qml.device("default.qubit", wires=1)

# Define a quantum node for the quantum layer
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    qml.RX(inputs[0], wires=0)
    qml.RY(weights[0], wires=0)
    qml.RZ(weights[1], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define a quantum layer using PennyLane's qnode
class QuantumLayer(nn.Module):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        # Trainable weights for the quantum circuit
        self.q_weights = nn.Parameter(torch.Tensor(2).uniform_(-np.pi, np.pi))

    def forward(self, x):
        # Only pass the first element of the feature vector to the quantum circuit
        x = x[:, :1]  # Use the first feature from the CNN output
        # Call the quantum circuit and pass the input and trainable weights
        q_out = torch.stack([quantum_circuit(x[i], self.q_weights) for i in range(x.shape[0])])
        return q_out


In [ ]:
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cnn = ClassicalCNN()  # Classical CNN
        self.quantum = QuantumLayer()  # Tiny quantum layer
        self.fc = nn.Linear(33, 2)  # Final fully connected layer for classification (32 CNN features + 1 quantum output)

    def forward(self, x):
        # Pass through the CNN
        cnn_output = self.cnn(x)  # Output shape: [batch_size, 32]

        # Pass a small portion of the CNN output to the quantum layer
        quantum_output = self.quantum(cnn_output)  # Output shape: [batch_size]

        # Ensure quantum output is in the same dtype (float32) as the CNN output
        quantum_output = quantum_output.float()  # Convert to float32

        # Expand quantum output to match CNN output dimensions
        quantum_output = quantum_output.unsqueeze(1)  # Shape becomes [batch_size, 1]

        # Concatenate the quantum output back to the CNN output
        combined_output = torch.cat([cnn_output, quantum_output], dim=1)  # Concatenating along feature dimension

        # Final classification layer
        x = self.fc(combined_output)  # Pass combined output through the final fully connected layer
        return x


In [ ]:
import torch.optim as optim

def train_model(model, train_loader, val_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation step
        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {correct/total:.4f}')

# Initialize and train the hybrid model
model = HybridModel()
train_model(model, train_loader, val_loader, epochs=10)


KeyboardInterrupt: 